In [ ]:
from sqlalchemy import *

In [ ]:
from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))
from pyspark.sql import functions as F, DataFrame
import datetime as dt
from datetime import date, datetime, timedelta
from dateutil.relativedelta import relativedelta
from pyspark.sql.window import Window
import pyspark.sql.types as t
from decimal import Decimal
from pyspark.sql.functions import regexp_replace
import os

In [ ]:
from dataproc_sdk.dataproc_sdk_datiopysparksession.datiopysparksession import DatioPysparkSession
datioSparkSession = DatioPysparkSession().get_or_create()

from dataproc_sdk.dataproc_sdk_datiopysparksession import datiopysparksession
dataproc = datiopysparksession.DatioPysparkSession().get_or_create()

from dataproc_sdk.dataproc_sdk_schema.datioschema import DatioSchema
from dataproc_sdk.dataproc_sdk_datiofilesystem.datiofilesystem import DatioFileSystem

In [ ]:
# para evitar problemas de tipología de datos
spark.conf.set("spark.sql.parquet.enableVectorizedReader", "false")

# Funciones

**hacer que la función no tome los csv para calcular la fecha, que se quede solo en el primer nivel de closing date**

In [ ]:
# # calculamos la fecha más reciente de la ruta tomando como campo de particion el pasado como parámetro
# def last_partition (path:str, campo:str):
#     window = Window.partitionBy().orderBy(F.col(campo).desc())
    
#     # opcion 1
#     df = dataproc.read().parquet(path
#                                ).withColumn("rn", F.row_number().over(window)
#                                ).where(F.col("rn") == 1).drop('rn')
    
#     # opcion 2
#     # df = dataproc.read().parquet(path).coalesce(1
#     #                                            ).orderBy(F.col(campo).asc()
#     #                                            ).groupBy().agg(F.last(campo).alias(campo))
    
#     fecha = [x[campo] for x in df.select(campo).collect()][0]
#     print('Fecha de datos:',fecha)
#     return fecha
    

In [ ]:
# calculamos la fecha más reciente de la ruta tomando como campo de particion el pasado como parámetro
def last_partition (p_path:str, campo:str):
    
    datio_path = DatioFileSystem().get().qualify(p_path)
    fs = datio_path.fileSystem()
    path = datio_path.path()
    path_list = fs.listStatus(path)
    paths = [path.getPath().toString() for path in path_list] #listado de todos los paths de la ruta pasada
    
    l_fechas = [element.split(campo+'=')[1] for element in paths if campo in element] #listado de todas las fechas
    return max(l_fechas) # fecha mayor

# Configuracion

In [ ]:
# Ruta Raiz en SBX
root_path = '/data/sandboxes/dslb/data/Joystick/TITULIZACIONES/cartera_optima/'

In [ ]:
mode_write= 'overwrite'# 'append'
# f_write = '2024-06-18' # particion de fechas

In [ ]:
f_write = last_partition (root_path, 'closing_date')
root_pathc = root_path + 'closing_date=' + str(f_write)
root_pathc

In [ ]:
# Rutas datos en SBX
path_facilities = root_pathc + '/facilities'

path_limites_only = root_pathc + '/limites'

path_constantes = root_pathc + '/constants'

path_excluidas = root_pathc + '/facilities_excluded'

path_facilities_total = root_pathc + '/cartera_titulizar' # foto con toda la traza de las operaciones, para poder seleccionar las columnas necesarias

In [ ]:
SCHEMA = 'joystick'

TABLES = {'joystick.securitization_limit':path_limites_only,
         'joystick.securitization_constant':path_constantes,
         'joystick.securitization_facilities_excluded':path_excluidas,
         'joystick.securitization_facilities_completed':path_facilities_total,
         # 'joystick.securitization_facilities:
         } 

In [ ]:
# INDICES = {'i_piloto': ['BEC_CDM_RELACIONES_POSTG','BEC_CDM_RELACIONES_CAPT_POSTG',
#                         'BEC_CDM_PILOTOS_POSTG','BEC_CDM_PILOTOS_CAPT_POSTG'],
#           'i_relacionada': ['BEC_CDM_RELACIONES_POSTG','BEC_CDM_RELACIONES_CAPT_POSTG',
#                             'BEC_CDM_RELACIONADAS_POSTG','BEC_CDM_RELACIONADAS_CAPT_POSTG'],
#           'i_oficina': ['BEC_CDM_PILOTOS_POSTG','BEC_CDM_PILOTOS_CAPT_POSTG'],
#           'i_gestor': ['BEC_CDM_PILOTOS_POSTG','BEC_CDM_PILOTOS_CAPT_POSTG'],}

## Postgres

In [ ]:
user = os.getenv("JPY_USER")
jpy = os.getenv("JPY_BASE_URL")
uuaa = jpy.split("/")[2]
vault = os.getenv("VAULT_HOSTS")
environ = vault.split(".")[3]
country = vault.split(".")[5]

In [ ]:
print('user:',user)
print('jpy:',jpy)
print('uuaa:',uuaa)
print('vault:',vault)
print('environ:',environ)
print('country:',country)

In [ ]:
# Define here the postgres host, port and database                        
#postgres_host = "ep-w-pgreporting.marathon.l4lb." + environ + ".daas." + country + ".igrupobbva:81"
postgres_host = "ep-w-pgreporting.sandbox.extservices.platform.marathon.mesos:5432"
postgres_db = uuaa
print(postgres_host + "/" + postgres_db)

# Path to needed certificates taken from Spark properties
sslKey = "/opt/sds/stratio_analytic/security/" + user + ".key"
sslCert = "/opt/sds/stratio_analytic/security/"  + user + ".pem"
sslRootCert = "/opt/sds/stratio_analytic/security/ca-bundle.pem"

pgurl = ("postgresql://{user}@{host}/{db}?sslmode=verify-full&sslcert={sslCert}&sslrootcert={sslRootCert}&sslkey={sslKey}"
               .format(sslCert=sslCert,
                       sslKey=sslKey,
                       sslRootCert=sslRootCert,
                       user=user,
                       host=postgres_host,
                       db=postgres_db))
pg = create_engine(pgurl)


sslCertdbc = spark.conf.get("spark.ssl.datastore.certPem.path")
sslKeydbc = spark.conf.get("spark.ssl.datastore.keyPKCS8.path")
sslRootCertdbc = spark.conf.get("spark.ssl.datastore.caPem.path")

pgjdbc = ("jdbc:postgresql://{host}/{db}?user={user}&ssl=true&sslmode=verify-full&sslcert={sslCert}&sslrootcert={sslRootCert}&sslkey={sslKey}&prepareThreshold=0"
               .format(sslCert=sslCertdbc,
                       sslKey=sslKeydbc,
                       sslRootCert=sslRootCertdbc,
                       user=user,
                       host=postgres_host,
                       db=postgres_db))

# Postgre

## Borramos Tablas

In [ ]:
for k,v in TABLES.items():
    try:
        pg.execute("drop table "+k)
        print('Tabla borrada:',k)
    except:
        None

In [ ]:
# Generamos el esquema donde se van a almacenar las tablas
try:
    pg.execute('CREATE SCHEMA ' + SCHEMA)
    print('Esquema generado:',SCHEMA)
except:
    None

## Generamos Tablas

In [ ]:
for k,v in TABLES.items():
    try:
        print('**** TABLA: ',k,'*****')
        print('**** path de datos: ',v,'*****')
        # leemos dataframe
        df = spark.read.parquet(v)
#         df.show(5,False)
        df.write.option("truncate","true").jdbc(url=pgjdbc,table=k,mode=mode_write)        
        print('numero de registros:',df.count())

    except:
        None

## Añadir Indices

In [ ]:
# for i in INDICES.keys():
#     pg.execute('DROP INDEX '+ i)
#     print('indice borrado ',i)

In [ ]:
# for k,v in TABLES.items():
#     try:
#         print('tabla:',k)
# #         if(k in INDICES['i_piloto']):
# #             pg.execute('CREATE INDEX i_piloto ON ' + k + '(cif_piloto)')
# #             print('indice i_piloto generado en tabla ',k)
            
# #         if(k in INDICES['i_relacionada']):
# #             pg.execute('CREATE INDEX i_relacionada ON ' + k + '(cif_relacionada)')
# #             print('indice i_relacionada generado en tabla ',k)
            
#         if(k in INDICES['i_oficina']):
#             pg.execute('CREATE INDEX i_oficina ON ' + k + '(oficina_desc_piloto)')
#             print('indice i_oficina generado en tabla ',k)
            
#         if(k in INDICES['i_gestor']):
#             pg.execute('CREATE INDEX i_gestor ON ' + k + '(gestor_piloto)')
#             print('indice i_gestor generado en tabla ',k)
#     except Exception:
#         print('Se ha producido un error')
#         pass
# #     except:
# #         print('Se ha producido un error')
# #         continue

In [ ]:
# sorted(spark.read.parquet(path_pilotos_postg_cap).columns)

# TEST

In [ ]:
# query = "SELECT * FROM joystick.concentration_risk_commitment_P WHERE group_id='G00000000001233'"
query = "SELECT * FROM joystick.securitization_limit"

## Primera fila

In [ ]:
pt = pg.execute(query)
pt.first()

## Todas las filas

In [ ]:
def rows_as_dicts(cursor):
    """convert tuple result to dict with cursor"""
    col_names = [i[0] for i in cursor.description]
    return [dict(zip(col_names, row)) for row in cursor]

cursor = pg.execute(query).cursor
result = rows_as_dicts(cursor)

for k in result:
    print(k)
# result

## Tablas origen

In [ ]:
df = spark.read.parquet('/data/sandboxes/dslb/data/Joystick/TITULIZACIONES/cartera_optima/closing_date='+f_write+'/limites')

In [ ]:
df.show(5,False)